In [33]:
import setup
setup.init_django()

In [34]:
from market.models import StockQuote

In [35]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [36]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

# filtering data
qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

4342

In [37]:
total = 0
for obj in qs:
    total += obj.close_price

In [38]:
total /qs.count()

Decimal('219.3440179410409949332105021')

In [39]:
# average prices for entire month
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 2, 'avg_price': Decimal('170.1696036020583190')}, {'company': 1, 'avg_price': Decimal('226.9706961957967545')}]>

In [41]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('time')[:count]) # grabbing orginal window
rolling_qs.reverse() # orginal time order

In [42]:
rolling_qs

[<StockQuote: StockQuote object (313484)>,
 <StockQuote: StockQuote object (313483)>,
 <StockQuote: StockQuote object (313482)>,
 <StockQuote: StockQuote object (313481)>,
 <StockQuote: StockQuote object (313480)>]

In [44]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 313484 234.6300 234.6300
2 313483 234.4250 234.5275
3 313482 234.5300 234.5283333333333333333333333
4 313481 234.5000 234.52125
5 313480 234.0700 234.4310


In [45]:
qs = StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))

In [ ]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=
            frame = RowRange(start=frame_start, end=0), # based on amount we want to get
        )
)    
